# Save courses to RDF

## Read data from Firestore

In [60]:
# init
from os import environ
environ['FIRESTORE_EMULATOR_HOST'] = 'localhost:8080'
environ['FIREBASE_AUTH_EMULATOR_HOST'] = 'localhost:8083'
environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../serviceAccountKey.json'

import firebase_admin
from firebase_admin import firestore, initialize_app, credentials

if not firebase_admin._apps:
    cred = credentials.ApplicationDefault()
    initialize_app(cred)

db = firestore.client()

In [61]:
# get a program
siit_ref = db.collection(u'programs').document(u'siit')
program = siit_ref.get().to_dict()
program['naziv']

'softversko inzenjerstvo i informacione tehnologije'

In [62]:
# get courses
courses_snapshot = siit_ref.collection(u'courses').stream()
courses = [course.to_dict() for course in courses_snapshot]

Helper Functions

In [63]:
from re import compile
def date_to_xsd_date(string_with_date):
    p = compile('(\d{2}).(\d{2}).(\d{4}).')
    day, month, year = p.search(string_with_date).groups()
    return f'{year}-{month}-{day}'

In [116]:
from rdflib import *
from Aiiso import Aiiso
from University import University
from rdflib.collection import Collection
g = Graph()

g.parse('../ontologije/university.owl', format='application/rdf+xml')
uni = Namespace('http://www.semanticweb.org/marija/ontologies/2021/university.owl')
g.bind('uni', uni)

g.bind('dc', DC)
g.bind('foaf', FOAF)

## Write program to rdf

In [98]:
program_node = URIRef(program['url'])

g.add((program_node, RDF.type, Aiiso.Programme))
g.add((program_node, uni.name, Literal(program['naziv'])))
g.add((program_node, uni.espb, Literal(program['espb'])))
g.add((program_node, uni.semesters, Literal(program['brojSemestara'])))
g.add((program_node, uni.ScientificField, Literal(program['naucnoStrucneOblasti'])))
g.add((program_node, uni.LevelOfStudy, Literal(program['nivoStudija'])))
g.add((program_node, uni.EducationalField, Literal(program['obrazovnoPolje'])))
g.add((program_node, uni.Title, Literal(program['zvanje'])))

<Graph identifier=N7f7183ab24da4cadb6cefffd2186664a (<class 'rdflib.graph.Graph'>)>

## Write courses to rdf

In [75]:
def addProfessors(course, course_node):
    professors = course['izvodjaciNastave']
    
    # professor_node = BNode()
    professor_node = URIRef(course['url'] + "/teachers")
    g.add((course_node, uni['hasTeachers'], professor_node))

    for prof in professors:
        list_item = BNode()

        g.add((professor_node, FOAF.Person, list_item))
        g.add((list_item, FOAF.name, Literal(prof['punoIme'])))
        g.add((list_item, FOAF.title, Literal(prof['zvanje'])))
        g.add((list_item, uni.typeOfTeaching, Literal(prof['vidNastave'])))

In [76]:
def addLiterature(course, course_node):
    literature = course['literatura']
    
    # literature_node = BNode()
    literature_node = URIRef(course['url'] + "/literature")
    g.add((course_node, uni['hasLiterature'], literature_node))

    for lit in literature:
        list_item = BNode()

        g.add((literature_node, uni.Literature, list_item))
        g.add((list_item, DC.creator, Literal(lit['autori'])))
        g.add((list_item, DC.title, Literal(lit['naziv'])))
        g.add((list_item, DC.date, Literal(lit['godina'])))
        g.add((list_item, DC.publisher, Literal(lit['izdavac'])))
        g.add((list_item, DC.language, Literal(lit['jezik'])))

In [77]:
def addEvaluation(course, course_node):
    evaluation = course['formiranjeOcene']
    
    evaluation_node = URIRef(course['url'] + "/evaluation")
    g.add((course_node, uni['hasEvaluation'], evaluation_node))

    for e in evaluation:
        list_item = BNode()

        g.add((evaluation_node, uni.Evaluation, list_item))
        g.add((list_item, uni.points, Literal(e['brojPoena'])))
        g.add((list_item, uni.required, Literal(e['obavezna'])))
        g.add((list_item, uni.preExamination, Literal(e['predispitna'])))
        g.add((list_item, uni.activity, Literal(e['predmetnaAktivnost'])))

In [78]:
for i, course in enumerate(courses[0:3]):
    course_node = URIRef(course['url'])

    g.add((course_node, RDF.type, Aiiso.Course))
    g.add((course_node, DC.title, Literal(course['naziv'])))
    g.add((course_node, DC.description, Literal(course['sadrzaj'])))
    g.add((course_node,DC.date, Literal(date_to_xsd_date(course['osnovneInformacije']), datatype=XSD.date)))
    g.add((course_node, DC.subject, Literal(course['naucnaOblast'])))
    g.add((course_node, DC.type, Literal(course['kategorija'])))

    g.add((course_node, uni.Purpose, Literal(course['cilj'])))
    g.add((course_node, uni.espb, Literal(course['espb'])))
    g.add((course_node, uni.Year, Literal(course['godina'])))
    g.add((course_node, uni.Result, Literal(course['ishod'])))
    g.add((course_node, uni.Methodology, Literal(course['metodologijaIzvodjenjaNastave'])))
    g.add((course_node, uni.Semester, Literal(course['semestar'])))
    
    addProfessors(course, course_node)
    addLiterature(course, course_node)
    addEvaluation(course, course_node)


## Write tests to rdf

In [117]:
def addAnswers(question, question_node, questionuri):
    answers = question['possibleAnswers']
    
    answers_node = URIRef(questionuri + "/answers")
    g.add((question_node, uni['hasAnswers'], answers_node))
    
    for a in answers:
#         answer_node = BNode()

        g.add((answers_node, uni.Answer, Literal(a)))
#         g.add((answer_node, uni.text, Literal(a)))

In [118]:
def addCorrectAnswers(correctAnswers, question_node, questionuri):
    
    answers_node = URIRef(questionuri + "/correctAnswers")
    g.add((question_node, uni['correctAnswer'], answers_node))
    
    for a in correctAnswers:
#         answer_node = BNode()

        g.add((answers_node, uni.Answer, Literal(a)))
#         g.add((answer_node, uni.text, Literal(a)))

In [119]:
def addQuestionsAndAnswers(test, test_node, testuri):
    questions_snapshot = db.collection(u'tests').document(test['id']).collection(u'questions').stream()
    answers_snapshot = db.collection(u'tests').document(test['id']).collection(u'answers').stream()

    questions = [question.to_dict() for question in questions_snapshot]
    correctAnswers = [answer.to_dict() for answer in answers_snapshot]

    questions_node = URIRef(testuri + "/questions")
    g.add((test_node, uni['hasQuestions'], questions_node))
    
    for i, q in enumerate(questions):
        
        questionuri = testuri + "/questions" + q['id']
        question_node = BNode()
#         list_item = URIRef(testuri + "/questions")
#         g.add((test_node, uni['hasQuestions'], question_node))

        g.add((questions_node, uni.Question, question_node))
        g.add((question_node, uni.text, Literal(q['text'])))
        g.add((question_node, uni.points, Literal(q['maxPoints'])))
        g.add((question_node, uni.DomainProblem, Literal(q['domainProblemName'])))   
        
        addAnswers(q, question_node, questionuri)
        addCorrectAnswers(correctAnswers[i]['correctAnswers'], question_node, questionuri)


In [120]:
#get tests
tests_snapshot = db.collection(u'tests').stream()
# tests = [test.to_dict() for test in tests_snapshot]

for test_snap in tests_snapshot:
    test = test_snap.to_dict()
    test['id'] = test_snap.id
    testuri = "http://www.sotis/tests/" + test['id']
    test_node = URIRef(testuri)
    
    g.add((test_node, RDF.type, uni.Test))
    g.add((test_node, uni.name, Literal(test['name'])))
    g.add((test_node, uni.points, Literal(test['maxPoints'])))
    
#     addQuestions(test, test_node, testuri)
    addQuestionsAndAnswers(test, test_node, testuri)

{'correctAnswers': ['true'], 'id': 'o2er3'}
{'id': 'oi6eoj', 'correctAnswers': ['true']}
{'correctAnswers': ['int(1)', 'int(2.7)', "int('3')"], 'id': 'umui8'}
{'id': '4gxhx', 'correctAnswers': ['yes']}
{'id': 'e74yo', 'correctAnswers': ['used to store multiple items in a single variable.']}
{'id': 'cn29v', 'correctAnswers': ['ordered', 'changeable', 'not allowing duplicates']}
{'id': '3j0tw', 'correctAnswers': ['letters', 'digits', 'undescores', 'dollar signs']}
{'id': 'uhqhfk', 'correctAnswers': ['true']}
{'id': 'zeenx', 'correctAnswers': ['true']}
{'id': 'x9ens', 'correctAnswers': ['a block of code designed to perform a particular task', 'executed when "something" invokes it (calls it)']}
{'id': 'tp9hk', 'correctAnswers': ['a number', 'a string', 'an object']}
{'id': 'ygo8l', 'correctAnswers': ['properties', 'methods']}


In [19]:
# for i, test in enumerate(tests):
#     testuri = "http://www.sotis/tests/" + str(i)
#     test_node = URIRef(testuri)
    
#     g.add((test_node, RDF.type, uni.Test))
#     g.add((test_node, uni.name, Literal(test['name'])))
#     g.add((test_node, uni.points, Literal(test['maxPoints'])))

In [121]:
g.serialize(format='turtle', destination="program-courses-2.ttl")
print(g.serialize(format='turtle'))

@prefix : <http://www.semanticweb.org/marija/ontologies/2021/university#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://purl.org/vocab/aiiso/schema#> .
@prefix uni: <http://www.semanticweb.org/marija/ontologies/2021/university.owl> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

schema:name rdfs:domain schema:Programme,
        :Domain,
        :Test .

<http://www.semanticweb.org/marija/ontologies/2021/university> a owl:Ontology ;
    owl:imports schema:,
        foaf: .

:DomainProblem a owl:Class ;
    rdfs:subClassOf :Domain .

:EducationalField a owl:Class ;
    rdfs:subClassOf schema:Programme .

:LevelOfStudy a owl:Class ;
    rdfs:subClassOf schema:Programme .

:Methodology a owl:Class ;
    rdfs:subClassOf schema:Course .

:Purpose a owl:Class ;
    rdfs:subClassOf schema:Course .

:Result a owl:Class ;
    rdfs:subClassOf schema:Course .

:Sci

In [100]:
def addQuestions(test, test_node, testuri):
    questions_snapshot = db.collection(u'tests').document(test['id']).collection(u'questions').stream()
    questions = [question.to_dict() for question in questions_snapshot]
    
    questions_node = URIRef(testuri + "/questions")
    g.add((test_node, uni['hasQuestions'], questions_node))
    
    for q in questions:
        questionuri = testuri + "/questions" + q['id']
        question_node = BNode()
#         list_item = URIRef(testuri + "/questions")
#         g.add((test_node, uni['hasQuestions'], question_node))

        g.add((questions_node, uni.Question, question_node))
        g.add((question_node, uni.text, Literal(q['text'])))
        g.add((question_node, uni.points, Literal(q['maxPoints'])))
        g.add((question_node, uni.DomainProblem, Literal(q['domainProblemName'])))   
        
        addAnswers(q, question_node, questionuri)
